### Model training

The filtered, calculated and encoded features can now be trained with appropriate models.

The following approaches are considered:
- Multi classification problem with clustered data
- Mutli regression problem with two outputs (longitude/latitude)

In terms of the data we have the following approaches:
- variable sequence length - can take all points in POLYLINE in consideration, zero-padd start of sequence if necessary
- fixed sequence length - take 5 points from beginning of POLYLINE and 5 points from end of polyline 


Algorithms:
- Long term short term NN (multi-class classification and regression)
    - able to handle variable sequence length, therefore the total trip POLYLINE can be used
- Random forest(only regression) 
    - can handle outliers well as dataset still contains outliers
    - runs efficiently on large data set

In [1]:
import tensorflow

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.optimizers import Adam

In [29]:
import pandas as pd 
import numpy as np 
import json
import preprocessing

Matplotlib is building the font cache; this may take a moment.


### 1) Random Forest - Multiclass

In [5]:
n_cluster=1000

In [6]:
train_data = pd.read_parquet(f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_train.parquet')
test_data = pd.read_parquet(f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_test.parquet')

In [8]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [30]:
train_data = preprocessing.create_fix_length_sequences(train_data, 10)

In [65]:
features_config_1 = ['N_COORDINATE_POINTS','TOTAL_DISTANCE_KM', 'START_POINT_LON','START_POINT_LAT','2013_10',
       '2013_11', '2013_12', '2013_7', '2013_8', '2013_9', '2014_1', '2014_2',
       '2014_3', '2014_4', '2014_5', '2014_6', '2014_7', '10.0', '12.0',
       '13.0', '14.0', '15.0', '18.0', '20.0', '21.0', '23.0', '25.0', '26.0',
       '27.0', '28.0', '33.0', '34.0', '35.0', '36.0', '38.0', '40.0', '42.0',
       '52.0', '53.0', '54.0', '56.0', '57.0', '58.0', '6.0', '60.0', '61.0',
       '63.0', '7.0', '9.0', 'OTHER', 'Cloudy', 'Foggy', 'Rainy', 'Sunny',
       'Windy', 'A', 'B', 'C', '16.0', '2014_10', '2014_11', '2014_12',
       '2014_8', '2014_9', '47.0', '49.0','START_SEQUENCE_STACK','STOP_SEQUENCE_STACK']
label_config_1 = ['cluster_label']

In [36]:
estimator = RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators=200, min_samples_split=0.001)

In [52]:
X = train_data[features_config_1]
y = train_data[label_config_1]'

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestCl
from sklearn.multioutput import MultiOutputRegressor
rf = RandomForestRegressor(random_state=0)
wrapper = MultiOutputRegressor(rf)
cv = RepeatedKFold(n_splits=10,random_state=1)
n_scores = cross_val_score(wrapper, X, y, scoring='mean_squared_error', cv=cv)

In [ ]:
model = Sequential()
model.add(Input(input_shape=(X.shape[0], X.shape[1])))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2), activation='linear')

model.compile(optimizer='adam', loss='mse', metrics=['MeanSquaredError', 'MeanAbsoluteError'])
model.add(Dense(n_outputs))
X_pred_train = model.fit(X_train)